In [1]:
# Update sklearn to prevent version mismatches
#!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
#!pip install joblib

In [3]:
import warnings
warnings.simplefilter('ignore')

# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
#Delete line, testing
df[["koi_disposition","koi_fpflag_nt","koi_fpflag_ss","koi_fpflag_co","koi_fpflag_ec"]]

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec
0,CONFIRMED,0,0,0,0
1,FALSE POSITIVE,0,1,0,0
2,FALSE POSITIVE,0,1,0,0
3,CONFIRMED,0,0,0,0
4,CONFIRMED,0,0,0,0
...,...,...,...,...,...
6986,FALSE POSITIVE,0,0,0,1
6987,FALSE POSITIVE,0,1,1,0
6988,CANDIDATE,0,0,0,0
6989,FALSE POSITIVE,0,0,1,0


# Select your features (columns)

In [6]:
# Set features. This will also be used as your x values.
#HELP are these the right columns, source data was hard to follow.
selected_features = df[['koi_disposition',"koi_fpflag_nt","koi_fpflag_ss","koi_fpflag_co",'koi_period', 'koi_time0bk', 'koi_slogg', 'koi_srad', 'koi_kepmag']]
selected_features

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_period,koi_time0bk,koi_slogg,koi_srad,koi_kepmag
0,CONFIRMED,0,0,0,54.418383,162.513840,4.467,0.927,15.347
1,FALSE POSITIVE,0,1,0,19.899140,175.850252,4.544,0.868,15.436
2,FALSE POSITIVE,0,1,0,1.736952,170.307565,4.564,0.791,15.597
3,CONFIRMED,0,0,0,2.525592,171.595550,4.438,1.046,15.509
4,CONFIRMED,0,0,0,4.134435,172.979370,4.486,0.972,15.714
...,...,...,...,...,...,...,...,...,...
6986,FALSE POSITIVE,0,0,0,8.589871,132.016100,4.296,1.088,14.478
6987,FALSE POSITIVE,0,1,1,0.527699,131.705093,4.529,0.903,14.082
6988,CANDIDATE,0,0,0,1.739849,133.001270,4.444,1.031,14.757
6989,FALSE POSITIVE,0,0,1,0.681402,132.181750,4.447,1.041,15.385


In [7]:
##HELP, I thought this was important.. doesn't seem so? 
#Referencing One_Hot-encoding exercise from class

In [8]:
# # Compare this with Pandas get_dummies()
# data = df['koi_disposition']

# selected_features = pd.concat([selected_features.drop('koi_disposition', axis=1), pd.get_dummies(selected_features['koi_disposition'])], axis=1)
# selected_features

# Create a Train Test Split

Use `koi_disposition` for the y values

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
#from tensorflow.keras.utils import to_categorical

#assign x and y values
# X = selected_features.drop("koi_disposition", axis=1)
# y = selected_features[["koi_disposition"]]
X = df.drop("koi_disposition", axis=1)
y = df[["koi_disposition"]]

#split training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=84, stratify=y)

In [10]:
print(X.shape, y.shape)

(6991, 40) (6991, 1)


In [11]:
X.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [12]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
3309,0,0,0,0,10.471619,0.000153,-0.000153,136.287700,0.01180,-0.01180,...,-78,4.083,0.182,-0.084,1.574,0.240,-0.331,282.09314,42.160271,14.486
5173,0,1,0,0,4.206070,0.000003,-0.000003,132.248423,0.00050,-0.00050,...,-343,3.736,0.432,-0.108,3.185,0.789,-1.465,298.47604,41.079910,11.684
6070,1,0,0,0,37.537859,0.001282,-0.001282,159.736400,0.02760,-0.02760,...,-253,4.429,0.065,-0.195,1.082,0.331,-0.142,289.21732,48.204311,15.769
2421,0,0,0,0,105.881767,0.000131,-0.000131,156.979350,0.00101,-0.00101,...,-79,3.757,0.078,-0.052,2.499,0.233,-0.311,291.49722,42.728481,10.775
6239,1,0,1,1,25.952291,0.000846,-0.000846,154.868900,0.02050,-0.02050,...,-197,4.547,0.032,-0.179,0.866,0.217,-0.078,293.70291,38.956291,15.999


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [13]:
# Scale your data
from sklearn.preprocessing import StandardScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [ ]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train_scaled, y_train)
predictions = model.predict(X_test)

In [15]:
#Help scores are low... 

print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.843410261300782
Testing Data Score: 0.8421052631578947


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [16]:
# Create the GridSearchCV model

from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [17]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... C=1, gamma=0.0001, score=0.829, total=   0.3s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] ................... C=1, gamma=0.0001, score=0.847, total=   0.3s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.6s remaining:    0.0s


[CV] ................... C=1, gamma=0.0001, score=0.828, total=   0.3s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.847, total=   0.3s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.836, total=   0.3s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.829, total=   0.3s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.847, total=   0.3s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.828, total=   0.3s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.847, total=   0.3s
[CV] C=1, gamma=0.001 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   14.4s finished


GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]},
             verbose=3)

In [18]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 10, 'gamma': 0.0001}
0.8691600142629475


# Save the Model

In [19]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'wendy.sav'
joblib.dump(model, filename)

['wendy.sav']